In [27]:
import numpy as np

from bokeh.io import output_notebook, show
import bokeh.plotting

import scipy.integrate
import scipy.optimize

import biocircuits
import colorcet

import matplotlib.pyplot as plt

interactive_python_plots = True
import os
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "1pbk8bdv6u5psm9k9u3o38d9iohudpemj63bekpdcrbnefl5la44"

In [ ]:
# Show Bokeh output inline
output_notebook()

# Sliders
beta_slider = bokeh.models.Slider(
    title="α",
    start=0,
    end=4,
    step=0.1,
    value=1,
    width=125,
    format=bokeh.models.CustomJSTickFormatter(code="return Math.pow(10, tick).toFixed(2)"),
)
gamma_slider = bokeh.models.Slider(
    title="γ",
    start=-3,
    end=0,
    step=0.1,
    value=0,
    width=125,
    format=bokeh.models.CustomJSTickFormatter(code="return Math.pow(10, tick).toFixed(2)"),
)
rho_slider = bokeh.models.Slider(
    title="α0",
    start=-6,
    end=0,
    step=0.1,
    value=-3,
    width=125,
    format=bokeh.models.CustomJSTickFormatter(code="return Math.pow(10, tick).toFixed(2)"),
)
n_slider = bokeh.models.Slider(title="n", start=1, end=5, step=0.1, value=3, width=125)

def repressilator_rhs(mx, t, beta, gamma, rho, n):
    """
    Returns 6-array of (dm_1/dt, dm_2/dt, dm_3/dt, dx_1/dt, dx_2/dt, dx_3/dt)
    """
    m_1, m_2, m_3, x_1, x_2, x_3 = mx
    return np.array(
        [
            beta * (rho + 1 / (1 + x_3 ** n)) - m_1,
            beta * (rho + 1 / (1 + x_1 ** n)) - m_2,
            beta * (rho + 1 / (1 + x_2 ** n)) - m_3,
            gamma * (m_1 - x_1),
            gamma * (m_2 - x_2),
            gamma * (m_3 - x_3),
        ]
    )


# Initial condiations
x0 = np.array([0, 0, 0, 1, 1.1, 1.2])

# Number of points to use in plots
n_points = 1000


# Solve for species concentrations
def _solve_repressilator(log_beta, log_gamma, log_rho, n, t_max):
    beta = 10 ** log_beta
    gamma = 10 ** log_gamma
    rho = 10 ** log_rho
    t = np.linspace(0, t_max, n_points)
    x = scipy.integrate.odeint(repressilator_rhs, x0, t, args=(beta, gamma, rho, n))
    m1, m2, m3, x1, x2, x3 = x.transpose()
    return t, m1, m2, m3, x1, x2, x3


t, m1, m2, m3, x1, x2, x3 = _solve_repressilator(
    beta_slider.value,
    gamma_slider.value,
    rho_slider.value,
    n_slider.value,
    100.0,
)

cds = bokeh.models.ColumnDataSource(
    dict(t=t, m1=m1, m2=m2, m3=m3, x1=x1, x2=x2, x3=x3)
)

p_rep = bokeh.plotting.figure(
    frame_width=500,
    frame_height=200,
    x_axis_label="t",
    y_axis_label="Concentration of species",
    x_range=[0, 100.0],
)

colors = bokeh.palettes.d3["Category20"][6]
m1_line = p_rep.line(source=cds, x="t", y="m1", line_width=2, color=colors[1])
x1_line = p_rep.line(source=cds, x="t", y="x1", line_width=2, color=colors[0])
m2_line = p_rep.line(source=cds, x="t", y="m2", line_width=2, color=colors[3])
x2_line = p_rep.line(source=cds, x="t", y="x2", line_width=2, color=colors[2])
m3_line = p_rep.line(source=cds, x="t", y="m3", line_width=2, color=colors[5])
x3_line = p_rep.line(source=cds, x="t", y="x3", line_width=2, color=colors[4])

legend_items = [
    ("m₁", [m1_line]),
    ("x₁", [x1_line]),
    ("m₂", [m2_line]),
    ("x₂", [x2_line]),
    ("m₃", [m3_line]),
    ("x₃", [x3_line]),
]
legend = bokeh.models.Legend(items=legend_items)

p_rep.add_layout(legend, "right")

# Build the layout
layout = bokeh.layouts.column(
    bokeh.layouts.row(
        beta_slider,
        gamma_slider,
        rho_slider,
        n_slider,
        width=575,
    ),
    bokeh.layouts.Spacer(height=10),
    p_rep,
)

# Set up callbacks
def _callback(attr, old, new):
    t, m1, m2, m3, x1, x2, x3 = _solve_repressilator(
        beta_slider.value,
        gamma_slider.value,
        rho_slider.value,
        n_slider.value,
        p_rep.x_range.end,
    )
    cds.data = dict(t=t, m1=m1, m2=m2, m3=m3, x1=x1, x2=x2, x3=x3)

beta_slider.on_change("value", _callback)
gamma_slider.on_change("value", _callback)
rho_slider.on_change("value", _callback)
n_slider.on_change("value", _callback)
p_rep.x_range.on_change("end", _callback)

# Build the app
def repressilator_app(doc):
    doc.add_root(layout)

show(repressilator_app)

Loading BokehJS ...

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=a8ac9c29-8cb3-49af-b4fb-19e112a1ec10&bokeh-absolute-url=http://localhost:50389&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:50389', method='GET', uri='/autoload.js?bokeh-autoload-element=a8ac9c29-8cb3-49af-b4fb-19e112a1ec10&bokeh-absolute-url=http://localhost:50389&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "/Users/shivaramakrishna/Library/Python/3.9/lib/python/site-packages/tornado/web.py", line 1713, in _execute
    result = await result
  File "/Users/shivaramakrishna/Library/Python/3.9/lib/python/site-packages/bokeh/server/views/autoload_js_handler.py", line 62, in get
    session = await self.get_session()
  File "/Users/shivaramakrishna/Library/Python/3.9/lib/python/site-packages/bokeh/server/views/session_handler.py", line 145, in get_session
    session = await self.application_context.create_session_if_needed(sessi